# Finding similar items

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Finding similar items") \
    .getOrCreate()

raw_df = spark.sql("SELECT * FROM default.job_summary")
display(raw_df.limit(3))

index job_link job_summary 959895 https://www.linkedin.com/jobs/view/project-engineer-infrastructure-construction-at-kiewit-3778292964 Requisition ID:
170521
Job Level:
Mid Level
Home District/Group:
Northern California District
Department:
Project Engineering
Market:
Water/Wastewater
Employment Type:
Full Time
Position Overview
Early to mid-level project controls/project engineering position. Support multiple projects with scheduling, contract administration, and cost controls.
District Overview
Kiewit’s Northern California District is one of Kiewit’s leading infrastructure districts specializing in various types of work in the heavy civil, power, transportation, and water resource markets. Although most of our work is regionally based in Northern California, we also pursue various projects across the west coast. Our Northern California district has been extremely successful and have over $600 million dollars of work and plan to grow this region by $200 million in 2022. We are looking for individuals who are passionate about infrastructure and committed to the construction industry and seek a long-term career.
Location
Kiewit’s Northern California District is one of Kiewit’s leading infrastructure districts specializing in various types of work in the heavy civil, power, transportation, and water resource markets. Although most of our work is regionally based in Northern California, we also pursue various projects across the west coast. We are looking for individuals who are passionate about infrastructure and committed to the construction industry rather than design.
Responsibilities
Read and understand plans and specifications. Visualize two-dimensional drawings in three dimensions.
Examine/inspect field conditions and identify problems, inaccuracies, and cost saving measures that arise or that may be encountered. Take corrective actions as needed.
Assist with the development and maintenance of the project master schedule using appropriate techniques and Primavera scheduling software with input from other project management personnel.
Assist with completing monthly Time Impact Analysis’ (TIAs)
Assist projects with compiling and management of short-term schedule development and reporting (90-day and 3-Week look ahead schedules)
Review new issue drawings and contract documents for change of conditions; quantify impact of changes and review with job teams.
Assist in the tracking of project change orders and development of projections.
Complete progress and quantity audits to validate reported progress.
Produce monthly reports on schedule and cost progress, trends, and forecasts.
Understand company and project quality control and quality assurance procedures.
Assist project managers with development of project procedures.
Travel to jobsites as necessary.
Qualifications
2+ years’ experience in Project Controls/Project Engineering.
Knowledge of project management software such as Microsoft Office Suite and Primavera.
Experience utilizing SAP is preferred, but not required.
Experience performing Monthly Schedule Updates and Time Impact Analysis’.
Ability to analyze project data and provide summary level project reports for management.
Ability to travel.
Ability to read, analyze and identify discrepancies on engineering plans, specifications, and contract documents.
Highly motivated, with a demonstrated passion for excellence and taking initiative.
Strong interpersonal, written, and verbal communication skills.
Team player with the ability to work independently to meet deadlines, goals and objectives.
Strong organization, time management, and attention to detail.
Strong work ethic, willing to do what it takes to get the job done right the first time.
Demonstrated commitment to ethics and integrity.
Other Requirements:
Regular, reliable attendance.
Work productively and meet deadlines timely.
Communicate and interact effectively and professionally with supervisors, employees, and others individually or in a team environment.
Perform

## Text tokenization and cleaning

In [0]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import DocumentAssembler, Finisher, Pipeline
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner, Stemmer
from pyspark.sql.functions import concat_ws

# Define the NLP pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("job_summary") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized") \
    .setLowercase(True)

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("clean_tokens") \
    .setCaseSensitive(False)

stemmer = Stemmer() \
    .setInputCols(["clean_tokens"]) \
    .setOutputCol("stem")

finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["clean_tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

# Apply to the data
pipeline = Pipeline(stages=[document_assembler, tokenizer, normalizer, stopwords_cleaner, stemmer, finisher])
model = pipeline.fit(raw_df)
cleaned_df = model.transform(raw_df)

cleaned_df = cleaned_df \
        .select("index", "job_summary", "clean_tokens") \
        .cache()

In [0]:
display(cleaned_df.limit(1))

index job_summary clean_tokens 959895 Requisition ID:
170521
Job Level:
Mid Level
Home District/Group:
Northern California District
Department:
Project Engineering
Market:
Water/Wastewater
Employment Type:
Full Time
Position Overview
Early to mid-level project controls/project engineering position. Support multiple projects with scheduling, contract administration, and cost controls.
District Overview
Kiewit’s Northern California District is one of Kiewit’s leading infrastructure districts specializing in various types of work in the heavy civil, power, transportation, and water resource markets. Although most of our work is regionally based in Northern California, we also pursue various projects across the west coast. Our Northern California district has been extremely successful and have over $600 million dollars of work and plan to grow this region by $200 million in 2022. We are looking for individuals who are passionate about infrastructure and committed to the construction industry and seek a long-term career.
Location
Kiewit’s Northern California District is one of Kiewit’s leading infrastructure districts specializing in various types of work in the heavy civil, power, transportation, and water resource markets. Although most of our work is regionally based in Northern California, we also pursue various projects across the west coast. We are looking for individuals who are passionate about infrastructure and committed to the construction industry rather than design.
Responsibilities
Read and understand plans and specifications. Visualize two-dimensional drawings in three dimensions.
Examine/inspect field conditions and identify problems, inaccuracies, and cost saving measures that arise or that may be encountered. Take corrective actions as needed.
Assist with the development and maintenance of the project master schedule using appropriate techniques and Primavera scheduling software with input from other project management personnel.
Assist with completing monthly Time Impact Analysis’ (TIAs)
Assist projects with compiling and management of short-term schedule development and reporting (90-day and 3-Week look ahead schedules)
Review new issue drawings and contract documents for change of conditions; quantify impact of changes and review with job teams.
Assist in the tracking of project change orders and development of projections.
Complete progress and quantity audits to validate reported progress.
Produce monthly reports on schedule and cost progress, trends, and forecasts.
Understand company and project quality control and quality assurance procedures.
Assist project managers with development of project procedures.
Travel to jobsites as necessary.
Qualifications
2+ years’ experience in Project Controls/Project Engineering.
Knowledge of project management software such as Microsoft Office Suite and Primavera.
Experience utilizing SAP is preferred, but not required.
Experience performing Monthly Schedule Updates and Time Impact Analysis’.
Ability to analyze project data and provide summary level project reports for management.
Ability to travel.
Ability to read, analyze and identify discrepancies on engineering plans, specifications, and contract documents.
Highly motivated, with a demonstrated passion for excellence and taking initiative.
Strong interpersonal, written, and verbal communication skills.
Team player with the ability to work independently to meet deadlines, goals and objectives.
Strong organization, time management, and attention to detail.
Strong work ethic, willing to do what it takes to get the job done right the first time.
Demonstrated commitment to ethics and integrity.
Other Requirements:
Regular, reliable attendance.
Work productively and meet deadlines timely.
Communicate and interact effectively and professionally with supervisors, employees, and others individually or in a team environment.
Perform work safely and effectively. Understand and follow oral and written instructions, including warn

## Sentence embedding

In [0]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import ArrayType, FloatType

# Apply HashingTF
hashing_tf = HashingTF(inputCol="clean_tokens", outputCol="tf", numFeatures=10_000)
feat_data = hashing_tf.transform(cleaned_df)

# Compute the TF-IDF values
idf = IDF(inputCol="tf", outputCol="tf_idf", minDocFreq=5)
df = idf.fit(feat_data).transform(feat_data).cache()

In [0]:
display(df.limit(1))

index job_summary clean_tokens tf tf_idf 959895 Requisition ID:
170521
Job Level:
Mid Level
Home District/Group:
Northern California District
Department:
Project Engineering
Market:
Water/Wastewater
Employment Type:
Full Time
Position Overview
Early to mid-level project controls/project engineering position. Support multiple projects with scheduling, contract administration, and cost controls.
District Overview
Kiewit’s Northern California District is one of Kiewit’s leading infrastructure districts specializing in various types of work in the heavy civil, power, transportation, and water resource markets. Although most of our work is regionally based in Northern California, we also pursue various projects across the west coast. Our Northern California district has been extremely successful and have over $600 million dollars of work and plan to grow this region by $200 million in 2022. We are looking for individuals who are passionate about infrastructure and committed to the construction industry and seek a long-term career.
Location
Kiewit’s Northern California District is one of Kiewit’s leading infrastructure districts specializing in various types of work in the heavy civil, power, transportation, and water resource markets. Although most of our work is regionally based in Northern California, we also pursue various projects across the west coast. We are looking for individuals who are passionate about infrastructure and committed to the construction industry rather than design.
Responsibilities
Read and understand plans and specifications. Visualize two-dimensional drawings in three dimensions.
Examine/inspect field conditions and identify problems, inaccuracies, and cost saving measures that arise or that may be encountered. Take corrective actions as needed.
Assist with the development and maintenance of the project master schedule using appropriate techniques and Primavera scheduling software with input from other project management personnel.
Assist with completing monthly Time Impact Analysis’ (TIAs)
Assist projects with compiling and management of short-term schedule development and reporting (90-day and 3-Week look ahead schedules)
Review new issue drawings and contract documents for change of conditions; quantify impact of changes and review with job teams.
Assist in the tracking of project change orders and development of projections.
Complete progress and quantity audits to validate reported progress.
Produce monthly reports on schedule and cost progress, trends, and forecasts.
Understand company and project quality control and quality assurance procedures.
Assist project managers with development of project procedures.
Travel to jobsites as necessary.
Qualifications
2+ years’ experience in Project Controls/Project Engineering.
Knowledge of project management software such as Microsoft Office Suite and Primavera.
Experience utilizing SAP is preferred, but not required.
Experience performing Monthly Schedule Updates and Time Impact Analysis’.
Ability to analyze project data and provide summary level project reports for management.
Ability to travel.
Ability to read, analyze and identify discrepancies on engineering plans, specifications, and contract documents.
Highly motivated, with a demonstrated passion for excellence and taking initiative.
Strong interpersonal, written, and verbal communication skills.
Team player with the ability to work independently to meet deadlines, goals and objectives.
Strong organization, time management, and attention to detail.
Strong work ethic, willing to do what it takes to get the job done right the first time.
Demonstrated commitment to ethics and integrity.
Other Requirements:
Regular, reliable attendance.
Work productively and meet deadlines timely.
Communicate and interact effectively and professionally with supervisors, employees, and others individually or in a team environment.
Perform work safely and effectively. Understand and follow oral and written instructions, incl

## Top frequent words

In [0]:
word_freq_df = df \
    .withColumn("word", explode(col("clean_tokens"))) \
    .groupBy("word").count() \
    .orderBy("count", ascending=False)

display(word_freq_df)

word,count
work,38071
experi,30272
manag,29272
requir,28679
team,24421
provid,22354
includ,21129
servic,21024
show,20522
care,17822


In [0]:
print("Total number of unique words %.d" % word_freq_df.count())

Total number of unique words 57721


## Modeling

In [0]:
pairs = df.alias("df1").crossJoin(df.alias("df2")) \
    .filter(col("df1.index") > col("df2.index")) \
    .select(
        col("df1.job_summary").alias("job_summary_1"),
        col("df2.job_summary").alias("job_summary_2"),
        col("df1.tf").alias("tf_1"),
        col("df2.tf").alias("tf_2"),
        col("df1.tf_idf").alias("tfidf_1"),
        col("df2.tf_idf").alias("tfidf_2")
    ) \
    .cache()

### Top similar pairs based on Jaccard distance between BOW

In [0]:
def jaccard_similarity(sv1, sv2):
    set1 = set(sv1.indices)
    set2 = set(sv2.indices)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    if union == 0:
        return 0.0
    return intersection / union

jaccard_udf = udf(jaccard_similarity, FloatType())

pairs_with_similarity = pairs \
    .withColumn("jaccard_similarity", jaccard_udf(col("tf_1"), col("tf_2"))) \
    .select("job_summary_1", "job_summary_2", "jaccard_similarity") \
    .orderBy(col("jaccard_similarity").desc()) \
    .cache()

display(pairs_with_similarity.limit(30))

job_summary_1 job_summary_2 jaccard_similarity Our values start with our people, join a team that values you!
We are the nation’s largest off-price retailer with over 2,000 stores, and a strong track record of success and growth. Our focus has always been bringing our customers a constant stream of high-quality brands and on-trend merchandise at extraordinary savings. All while providing a fun and exciting treasure hunt experience.
As Part Of Our Team, You Will Experience
Success. Our winning team pursues excellence while learning and evolving
Career growth. We develop industry leading talent because Ross grows when our people grow
Teamwork. We work together to solve the hard problems and find the right solution
Our commitment to Diversity, Equity & Inclusion, and our community. We celebrate the backgrounds, identities, and ideas of those who work and shop with us because our differences make us stronger. We strive to be a positive force in our community.
Our Corporate headquarters are in Dublin, CA, we have 3 buying offices in key markets in New York City, Los Angeles, and Boston, and 7 distribution centers nationwide. With 2022 revenues of $18.7 billion, we are a Fortune 500 company who is committed to providing an inclusive work environment with continuous learning opportunities and development for our teams.
Come see what’s in store: https://jobs.rossstores.com/creative/about
General Purpose
The Retail Associate is responsible for ensuring our Customers have a positive shopping experience. The Associate makes eye contact, smiles, and greets all Customers in a courteous and friendly manner, treats fellow Associates with respect, efficiently follows all company best practice standards as outlined in all work centers, maintains a clean work area at all times, and ensures proper merchandise presentation. The Retail Associate is expected to be engaged in these tasks as assigned during all working hours and will be expected to perform a range of functions in all areas of the Store as business needs require. The Associate may be requested to perform additional tasks in specific situations, if performance of these tasks will help achieve our Customer service and operational goals.
Essential Functions
Understands that safety is the number one priority and practices safe behaviors in everything they do.
Maintains a high level of awareness on the sales floor to create a safe and secure shopping environment for everyone. Maintains clutter free, clear egress to emergency exits. Immediately corrects or reports any unsafe conditions or practices to Store Leadership.
Treats all Customers and Associates with respect and courtesy; is friendly and professional at all times. Recognizes fellow Associates using Company recognition programs.
Assists Customers in any way necessary - is register-trained, assists Customers with merchandise, and answers Customer questions in a polite and knowledgeable manner. Greets all Customers by making eye contact, smiling and saying "hello" throughout the Store as well as saying "thank you" with every register transaction.
Provides prompt and efficient responses to Customers at all times. Responds to Customer Service calls immediately. Handles all Customer issues in a courteous and helpful way, calling a member of the Store Leadership when needed.
Represents and supports the Company brand at all times.
Performs cleaning tasks such as sweeping, mopping, dusting, wiping counters and mirrors, replenishing restroom supplies, and disinfecting high-touch areas including the front-end, sales floor and restrooms to help maintain merchandise, customer and Team areas.
Maintains a professional appearance and adheres to the Company's dress code at all times.
Performs cleaning tasks such as sweeping, mopping, dusting, wiping counters and mirrors, and replenishing restroom supplies to help maintain merchandise, customer and Team areas.
Performs daily assigned sizing and recovery per company best practice to ensure a neat, cl

### Top similar pairs based on Cosine similarity between TF-IDF vectors

In [0]:
def cosine_similarity(v1, v2):
    dot_product = float(v1.dot(v2))
    norm_v1 = float(v1.norm(2))
    norm_v2 = float(v2.norm(2))
    return dot_product / (norm_v1 * norm_v2)

cosine_similarity_udf = udf(cosine_similarity, FloatType())

pairs_with_similarity = pairs \
    .withColumn("cosine_similarity", cosine_similarity_udf(col("tfidf_1"), col("tfidf_2"))) \
    .select("job_summary_1", "job_summary_2", "cosine_similarity") \
    .orderBy(col("cosine_similarity").desc()) \
    .cache()

display(pairs_with_similarity.limit(30))

job_summary_1 job_summary_2 cosine_similarity Our values start with our people, join a team that values you!
We are the nation’s largest off-price retailer with over 2,000 stores, and a strong track record of success and growth. Our focus has always been bringing our customers a constant stream of high-quality brands and on-trend merchandise at extraordinary savings. All while providing a fun and exciting treasure hunt experience.
As Part Of Our Team, You Will Experience
Success. Our winning team pursues excellence while learning and evolving
Career growth. We develop industry leading talent because Ross grows when our people grow
Teamwork. We work together to solve the hard problems and find the right solution
Our commitment to Diversity, Equity & Inclusion, and our community. We celebrate the backgrounds, identities, and ideas of those who work and shop with us because our differences make us stronger. We strive to be a positive force in our community.
Our Corporate headquarters are in Dublin, CA, we have 3 buying offices in key markets in New York City, Los Angeles, and Boston, and 7 distribution centers nationwide. With 2022 revenues of $18.7 billion, we are a Fortune 500 company who is committed to providing an inclusive work environment with continuous learning opportunities and development for our teams.
Come see what’s in store: https://jobs.rossstores.com/creative/about
General Purpose
The Retail Associate is responsible for ensuring our Customers have a positive shopping experience. The Associate makes eye contact, smiles, and greets all Customers in a courteous and friendly manner, treats fellow Associates with respect, efficiently follows all company best practice standards as outlined in all work centers, maintains a clean work area at all times, and ensures proper merchandise presentation. The Retail Associate is expected to be engaged in these tasks as assigned during all working hours and will be expected to perform a range of functions in all areas of the Store as business needs require. The Associate may be requested to perform additional tasks in specific situations, if performance of these tasks will help achieve our Customer service and operational goals.
Essential Functions
Understands that safety is the number one priority and practices safe behaviors in everything they do.
Maintains a high level of awareness on the sales floor to create a safe and secure shopping environment for everyone. Maintains clutter free, clear egress to emergency exits. Immediately corrects or reports any unsafe conditions or practices to Store Leadership.
Treats all Customers and Associates with respect and courtesy; is friendly and professional at all times. Recognizes fellow Associates using Company recognition programs.
Assists Customers in any way necessary - is register-trained, assists Customers with merchandise, and answers Customer questions in a polite and knowledgeable manner. Greets all Customers by making eye contact, smiling and saying "hello" throughout the Store as well as saying "thank you" with every register transaction.
Provides prompt and efficient responses to Customers at all times. Responds to Customer Service calls immediately. Handles all Customer issues in a courteous and helpful way, calling a member of the Store Leadership when needed.
Represents and supports the Company brand at all times.
Performs cleaning tasks such as sweeping, mopping, dusting, wiping counters and mirrors, replenishing restroom supplies, and disinfecting high-touch areas including the front-end, sales floor and restrooms to help maintain merchandise, customer and Team areas.
Maintains a professional appearance and adheres to the Company's dress code at all times.
Performs cleaning tasks such as sweeping, mopping, dusting, wiping counters and mirrors, and replenishing restroom supplies to help maintain merchandise, customer and Team areas.
Performs daily assigned sizing and recovery per company best practice to ensure a neat, cle

### K-Means clustering 100

In [0]:
from pyspark.ml.clustering import BisectingKMeans

bkm_100 = BisectingKMeans(featuresCol="tf_idf", predictionCol="cluster", k=100)

clusters_100 = bkm.fit(df).transform(df).select("cluster", "index", "job_summary")

In [0]:
cluster_count = clusters_100.groupby("cluster").count().orderBy("count", ascending=False)
display(cluster_count)

cluster,count
8,283
9,221
75,142
15,135
10,133
14,132
11,119
74,115
200,97
20,91


**Clusters with 100+ samples** seem to have more similar but still quite different job offerings

In [0]:
display(clusters_100.filter("cluster == 10"))

cluster,index,job_summary
10,746517,"What are we looking for in our Sr Claim Examiner- WC? Russell Tobin is seeking a Sr. Claim Examiner in Hauppauge, New York. Job title: Sr. Claim Examiner Rate: 37.56 Job Type: Contract Location: Hauppauge, New York Job Responsibilities: Interprets and makes decisions using independent judgment on more complex and unusual policy coverages and determines if coverages apply to claims submitted. Manages all aspects of investigative activity on complex claims. Directs the discovery and litigation strategy with legal counsel. Analyzes claims activity and prepares reports for clients/carriers and management. Establish reserves, using independent judgment and expertise and authorizes payments within scope of authority. settling claims in the most cost-effective manner and ensuring timely issuance of disbursements. Analyzes claims activities and prepares reports for clients, carriers and/or management Participates in claim reviews. Skills: Ability to work independently while assimilating various technical subjects. Strong written and oral communication, negotiation, and presentation skills. Advanced analytical and problem-solving skills, with the ability to manage and prioritize multiple projects. Show more Show less"
10,857201,"Job Description Job description Job Opportunity : NP with Wound Care experience Private Wound Care Practice is looking to Hire an NP with wound care experience. Requirements: 3 years + as a NP NP license Experience with Wound Care Experience with Debridement Experience with Grafting (big plus) Day to Day Responsibilities: Visiting patients to treat them off site post grafting etc.. Must be willing to travel (within Brooklyn/Bronx/Queens) Must have own car (gas/travel allowance is available) Salary & Benefits: $125-130k Base Plus strong bonus pay package. Target Compensation: $ 225-275k (Base + Bonus) Health, Insurance included. Job Type: Full-time Salary: $225,000.00 - $275,000.00 per year Benefits: 401(k) Dental insurance Health insurance Paid time off Medical specialties: Dermatology Geriatrics Medical-Surgical Plastic Surgery Wound Care Standard shift: Day shift Supplemental pay types: Bonus opportunities Completion bonus Weekly schedule: Monday to Friday Work setting: In-person Multiple locations Private practice Ability to Relocate: Brooklyn, NY: Relocate before starting work (Required) Work Location: In person Company Description Long term growth opportunity Long term growth opportunity Show more Show less"
10,1034702,"Whether you are searching for a position in your area or in another state, we have professionals to help you achieve your goals through our relationships with facilities nationwide - in rural settings, small cities, and major metropolitan areas. Your CompHealth consultant can help you find a position with the salary and perks that meet your expectations and match your training, experience, and goals. We have many exciting opportunities available, as we are one of the largest and most reputable staffing firms in the United States. Our consultants are known for their expertise, so when a CompHealth consultant represents you, our clients pay attention. Contact Shari Berman shari.berman@comphealth.com (203) 663-9370. Treat a geriatric patient base Outpatient No call See as many as 12 patients per day 40-hour work week Guaranteed salary Comprehensive benefits Residents and practicing physicians are welcome to apply Our services are free for you We help negotiate your salary and contract We coordinate interviews and help with licenses Specialized recruiters match your career preferences Experienced support teams take care of every detail Show more Show less"
10,969576,"Responsibilities We are seeking a Full-Time Family Medicine Physician to join their team providing exceptional care in the ambulatory setting. The group currently consists of 2 physicians. This practice utilizes the EPIC EMR, which is universal throughout the hospital. Highlights Strong support 

**Clusters with less than 30 samples** finally have some meaningful results and indeed collect very similar job offerings.

In [0]:
display(clusters_100.filter("cluster == 29"))

cluster,index,job_summary
29,807818,"Project Manager Civil Engineering Contractor North England The ideal candidate will be an established civil engineering Project Manager working with either a Tier 1 or 2 Contractor on schemes up to £15 million. You will be 'hands on' and be able to confidently deliver a range of civil engineering schemes including earthworks, S278, drainage, junction improvements, FRC works as well as some structures. Our client has a national remit with projects throughout the UK. Initially we are looking for candidates to deliver schemes in South Yorkshire and the North East. Responsibilities Management of Site Team including, Site Manager, Works Manager and Engineer Working Autonomously reporting to a visiting Contract Manager Liaison with Commercial Team and Senior Management Preparation of Risk Assessments and Method Statements Management of Sub Contractors Placement of material orders in a timely fashion Update site plan to reflect delivery About You You will be a skilled Civil Engineering Project Management professional with experience of working with SME contractors. You will have likely worked on numerous projects for the same contractor. You will be looking for long term career opportunity working with a rapidly expanding SME contractorCalibre Search promote equality in the workplace and we welcome applications from all suitably skilled or qualified candidates regardless of their sex, race, disability, religion/beliefs, sexual orientation or age. We act as both an employment agency and employment business Show more Show less"
29,819723,"Technical Project Engineer / Manager Location : Based at Challenger Handling Limited in Hull Challenger Handling Limited is part of a group of engineering companies which has recently converted to an Employee Ownership Trust. It is now looking to strengthen its management team. Responsibilities Working closely with the management team to becomethe customer interface, provide technical back-up and manage projects such as recycling equipment and systems, clinical waste processing and sterilisation plant, timber handling systems and other special projects. This will include time on site, which is generally in the UK but it can be Worldwide as needed. The successful applicant must be an engineer with good commercial acumen and the ability to deal with people from all walks of life. They must be self-motivated, willing to take on responsibility with minimal supervision and be able to problem solve in a fast moving, competitive, commercial environment. Ideally, a degree qualified mechanical engineer working as a Technical or Project Engineer / Manager for an equipment manufacturer but the candidate must be SolidWorks, SAGE, MS Word and MS Excel literate. The position is based at the Hull office so candidates MUST live within a one-hour commute. There will be some home working but there must be a willingness to travel extensively throughout the UK and abroad as required. Remuneration : Salary range between £ 35,000 50,000 subject to abilityand experience plus car (or car allowance) and regular bonuses STRICTLY NO AGENCIES PLEASE Show more Show less"
29,146106,"Job Summary We are currently seeking a highly skilled Maintenance Manager to oversee all aspects of facility management for Aerospace in Commerce, CA. The successful candidate will be responsible for maintaining the physical integrity of our facility and ensuring a safe, secure, and efficient work environment for our employees. This role will manage a crew of 8 people across all three shifts to handle maintenance and operations. Key Responsibilities Supervise a team of 8 maintenance staff across three shifts to ensure optimal productivity and performance. Oversee all aspects of facility maintenance, including HVAC, electrical systems, plumbing, and general repairs. Coordinate and manage machinery maintenance schedules to ensure minimal downtime and optimal performance. Maintain compliance with state and federal regu

In [0]:
display(clusters_100.filter("cluster == 45"))

cluster,index,job_summary
45,127004,"Specific Duties To Perform Lead in the development and design of automation projects using OutSystems Multi-Experience Development Platform Writing and maintaining code Create data migration tools to migrate information from legacy systems Web application development, process design, and project delivery practices Working on critical bug fixes Conduct code reviews Enhance some of the OutSystems local applications through development of change requests, investigation and implement fixes on issues raised. Responsible for providing expert level assistance to our client using the OutSystems platform to run their banking applications. Exhibits expert business knowledge and leverages strong customer relationships. Lead and guide junior and mid-level developers Position Qualifications/Experience/Education: Bachelor's degree in a relevant field and 5+ years of experience OutSystems experience of at least two (2) years Programming experience Knowledge of databases; NoSQL, MySQL, MongoDB etc. Knowledge of APIs Knowledge of HTML, CSS and JavaScript Knowledge on .NET C# Deep knowledge in web-based languages (JavaScript, JQuery, CSS, HTML) Proficiency with Oracle SQL and ASP.NET Knowledge of web services (REST and SOAP) Desired Position Qualifications: OutSystems Certification: Web Associate Developer or Higher Expert with OutSystems Service Studio & Integration Studio Clearance Requirement: Eligible to obtain a Public Trust Clearance Location : This position is fully remote within the United States. Show more Show less"
45,176520,"Role Purpose/Summary: Iyuno is currently seeking an experienced and capable Systems Administrator accustomed to working in a fast-paced environment. This role will be responsible for maintaining network, storage and server systems in a corporate production environment. The position will prioritize in maintaining our network infrastructure and will assist with storage and server systems in a 24x7x365 environment. This position will provide first tier support for business unit related issues. This individual will report to the Manager of Infrastructure and will possess superior organizational skills and sound judgment. The ideal candidate will be a self-starter, detailed-oriented, personable with a team-player mind set. Responsibilities: Troubleshoot all aspects of software and hardware related desktop/laptop problems Troubleshoot desktop networking problems Provide training on software and hardware usage Prioritize, coordinate, execute and complete technology projects on time Document configurations and perform change management procedures Consult with IT Management on current and future computing needs of company Support overall content protection efforts Ability to be “on call” for after hours and weekend coverage support Ability to work flexible hours (i.e. overtime, weekends, etc.) to provide emergency coverage in a mission critical 24/7/365 environment as needed Performs other related duties, as assigned by IT management Skills and Experience Required: Excellent communication skills Excellent negotiation skills Good technical understanding Ability to quickly grasp complex subject matters Ability to work to tight deadlines and under pressure Self-motivation Proactivity Ability to work independently and on own initiative Ability to be an excellent team player Good time management skills Excellent organizational skills Strong interpersonal skills Good attention to detail Excellent problem-solving skills Enthusiastic and willing to learn Essential Qualifications: Bachelor degree in computer science or related degree preferred and/or four years related experience. Experience with Windows is required. Macintosh and Linux is a plus Experience with backup / archive applications. VMWare \ Virtualization, Office 365, Google Suite, Aspera, Signiant is a plus Knowledge of film and broadcast technologies a plus Previous experience in the entertainment industry is a plus Show more Show

### K-Means clustering 1000

In [0]:
from pyspark.ml.clustering import BisectingKMeans

bkm_1000 = BisectingKMeans(featuresCol="tf_idf", predictionCol="cluster", k=1000)

# Fit the model
clusters_1000 = bkm_1000 \
    .fit(df) \
    .transform(df) \
    .select("cluster", "index", "job_summary")

In [0]:
cluster_count = clusters_1000.groupby("cluster").count().orderBy("count", ascending=False)
display(cluster_count)

cluster,count
8,283
9,221
75,142
15,135
10,133
14,132
11,119
74,115
200,97
20,91


In [0]:
display(clusters_1000.filter("cluster == 8"))

cluster,index,job_summary
8,68105,"RNnetwork has an excellent travel opportunity for a provider experienced in the following specialty: Technologist-Certified Surgical Technologist. This opportunity will be available in Portland, ME. The Surgical position will cover and provide care within that unit. The facility needs an experienced provider to help care for their patients. Your schedule or shifts for this assignment will be: 7:00 PM - 07:00 AM. Travel nursing is a rewarding and exciting career path, and RNNetwork has the best recruiters and support in the industry. We do our very best to get you the pay, location, schedule, and setting you want. Contact us today to find your next travel assignment! Specialty: Registered Nurse - Surgical Shift: 7:00 PM - 07:00 AM Customized benefits package with the pay and insurance coverage you need, beginning day one Paid private housing and utilities, and reimbursement for travel Reimbursement for licensing and certification Why choose RNnetwork? Show more Show less"
8,393210,"IMCS GROUP is seeking a local contract MRI Technologist for a local contract job in New York, New York. Job Description & Requirements Specialty: MRI Technologist Discipline: Allied Health Professional Start Date: 01/31/2024 Duration: 13 weeks 40 hours per week Shift: 8 hours, evenings Employment Type: Local Contract ARRT MRI certified, NYS DOH Radiography license to inject, AHA BLS. Proficiency with GE MRI Platform. Preferred but not required: proficiency with GE RIS/UV and Epic Computer programs **Tech 1 Show more Show less"
8,439587,"Training on 1st shift for up to a few months – 2nd shift schedule is Mon – Thurs 3:30 p.m. – 2 a.m. Polish stainless steel components with a variety of abrasives, driven by electric and air power tools. For more information on these positions visit our site: www.abccorporate.com OR apply at jobs@alleghenybradford.com Allegheny Bradford Corp. is an equal opportunity employer and administers all personnel practices without regard to race, color, religion, sex, age, national origin, disability, sexual orientation, gender identity or expression, marital status, veteran status, genetics or any other category protected under applicable law Show more Show less"
8,351477,"Are you an outgoing people-person who thrives in a fast-paced work environment? If so, we are looking for you! Our lending team is seeking a junior loan officer to help applicants navigate through the home loan process. We offer the training, the technology, and the team support to help you succeed. If you are looking for a lucrative career using your communications and interpersonal relationship-building talents, apply today! Compensation: $62,500 at plan earnings Show more Show less"
8,322445,Brightstar Care in conjunction with Caring Touch Healthcare Academy is looking for RNs to teach at the affiliate CNA school...Caring Touch Healthcare Academy. The hours are flexible and can work with most schedules. This can be either a full time position or a part time position. Compensation Information $0.0 / - $0.0 / Starting At: 0.0 Up To: 0.0 Show more Show less
8,724284,Prospección de clientes. Colocación de productos. Atención de calidad hacia los clientes. Disposición de tiempo. Manejo de redes sociales Ganas de trabajar. Mostrar más Mostrar menos
8,1082609,"Job Description: Position Type: Travel Nursing Specialty: Nursing RN ER/Trauma Experience: Candidates must have a minimum of 1 year of similar ER/Trauma-RN experience to be considered. Duration: 13 Weeks Location: Saratoga Springs, NY 12866 Pay Package: $2,472.00 total gross weekly for 36 hours per week for 13 weeks. Stipends:$1,295.00 weekly Taxable rates: Regular hours: $32.68 Gap hours: $68.65 Overtime hours: $77.39 Benefits: First Day Medical/Dental/Vision Great Pay! 401(k) Sick Leave Weekly Payroll Show more Show less"
8,582378,"Job Description Clawson & Staubes is hiring Civil Defense Litigation Attorneys in Charleston, Myrtle Beach, Columbia, Greenville, and Ch

In [0]:
display(clusters_1000.filter("cluster == 222"))

cluster index job_summary 222 291811 WE ARE EPIC! ARE YOU?
Join our outstanding team! Unlock limitless professional possibilities with EPIC STAFFING GROUP!
Looking for a workplace that inspires, challenges, and ignites your passion? Look no further! At EPIC, we believe in rewriting the rules and making every workday an extraordinary adventure. We're not just another company; we're a force of innovation, collaboration, and sheer brilliance.
Picture yourself surrounded by top-notch talent, fueled by boundless curiosity, and empowered to make a genuine impact on the world. Here, you'll find a vibrant ecosystem where your ideas are valued, your growth is nurtured, and your creativity takes flight.
At EPIC, we don't settle for the ordinary; we strive for the extraordinary. Every day offers an opportunity to redefine what's possible. Join us as we reshape healthcare and life sciences staffing, break boundaries, and create a future that surpasses imagination.
Get ready to immerse yourself in a culture that celebrates diversity, fosters innovation, and nurtures your personal and professional aspirations. With a team of like-minded visionaries and a supportive network, your potential will soar to new heights.
If you're ready to embark on an extraordinary journey, where innovation meets passion and challenges breed growth, then EPIC is the place for you. Come be a part of something truly remarkable and write your own success story with us!
Join our EPIC team and let your brilliance shine!
About the Position:
Epic Specialty Staffing, Diagnostic Imaging Division, is looking for an Operations Specialist who will be responsible for all aspects of moving radiologists through the process from credentialing to providing services. This position will be located onsite at the office in South Barrington, Illinois.
About Us:
Epic Specialty Staffing, Diagnostic Imaging Division offers staffing solutions focused on providing remote radiologists who can seamlessly “plug in” to a radiology practice or hospital to provide after hours or surge-support to provide remote reading services.
What You'll Do:
Perform all aspects of ensuring that the radiologists have all the resources needed to begin providing services.
Maintain client and radiologist schedules.
Lead client meetings to facilitate radiologists preparing to provide services.
Handle advanced operational tasks and communication.
Manage incoming and outgoing communications between multiple parties while meeting policy timelines and updating the client and radiologist schedules.
Provide status reports and serve as a conduit between client and radiologist, and vice versa.
Document and log radiologist productivity.
Who You Are:
You are detail oriented and have the ability to complete assignments with a high degree of accuracy and in a timely and efficient manner.
You are flexible and adaptive to changing priorities and service needs.
With exceptional verbal and written communication skills, you have the ability to exchange non-routine information using tact and persuasion.
What You've Done:
Graduated from high school or possess an educational equivalent.
Worked with Microsoft Office.
Preference to individuals who have work experience in a similar role; experience working for a medical company; experience in a task-based role for a small-midsized organization.
Salary: $30/hour
Operations Specialist Full Job Description
What We Offer:
Vacation and Time Off
Generous PTO Policy
Paid Parental Leave
Twelve Annual Paid Holidays
Office Life and Perks
Active Employee Engagement Programs
Cellphone Stipend
Technology Stipend (for hybrid/remote positions)
Quarterly Snack Boxes
Diversity, Equity, Inclusion, and Belonging Program
Remote Work
Professional Development
Charitable Matching
Health and Wellness
Medical, Dental, Vision, and Hearing Benefits
Wellbeats Health and Wellness Programs
Employee Assistance Program (EAP)
Life and Disability Insurance
Pet Insurance
Financial and Retirement
401K
FSA and HSA Plans
Fin